In [1]:
import torchvision.models
import torch

In [2]:
import torchinfo

In [3]:
# a = torchvision.models.mobilenet_v3_small(torchvision.models.MobileNet_V3_Small_Weights.DEFAULT)
# b = torchvision.models.shufflenet_v2_x1_0(torchvision.models.ShuffleNet_V2_X1_0_Weights.DEFAULT)
# b = torchvision.models.shufflenet_v2_x0_5(torchvision.models.ShuffleNet_V2_X0_5_Weights.DEFAULT)
c = torchvision.models.mnasnet0_5(torchvision.models.MNASNet0_5_Weights.DEFAULT)

# resnet18_model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)


/home/nicholassv/anaconda3/envs/thesis_linux_3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/mnasnet0.5_top1_67.823-3ffadce67e.pth" to /home/nicholassv/.cache/torch/hub/checkpoints/mnasnet0.5_top1_67.823-3ffadce67e.pth
100%|██████████| 8.59M/8.59M [00:02<00:00, 3.49MB/s]


In [5]:
b.conv1[0] = torch.nn.Conv2d(2, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
b.fc = torch.nn.Identity()

In [18]:
# a.classifier = torch.nn.Identity()
# a.features[0][0] =  torch.nn.Conv2d(2, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

In [4]:
c

MNASNet(
  (layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(8, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (8): Sequential(
      (0): _InvertedResidual(
        (layers): Sequential(
          (0): Conv2d(8, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(24, 24,

In [5]:

torchinfo.summary(c, input_size=(32*15,3,224,224))

/home/nicholassv/anaconda3/envs/thesis_linux_3.9/lib/python3.9/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/home/nicholassv/anaconda3/envs/thesis_linux_3.9/lib/python3.9/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                        Output Shape              Param #
MNASNet                                       [480, 1000]               --
├─Sequential: 1-1                             [480, 1280, 7, 7]         --
│    └─Conv2d: 2-1                            [480, 16, 112, 112]       432
│    └─BatchNorm2d: 2-2                       [480, 16, 112, 112]       32
│    └─ReLU: 2-3                              [480, 16, 112, 112]       --
│    └─Conv2d: 2-4                            [480, 16, 112, 112]       144
│    └─BatchNorm2d: 2-5                       [480, 16, 112, 112]       32
│    └─ReLU: 2-6                              [480, 16, 112, 112]       --
│    └─Conv2d: 2-7                            [480, 8, 112, 112]        128
│    └─BatchNorm2d: 2-8                       [480, 8, 112, 112]        16
│    └─Sequential: 2-9                        [480, 16, 56, 56]         --
│    │    └─_InvertedResidual: 3-1            [480, 16, 56, 56]         920
│    │    └─_Inv

In [1]:
import torch
import torch.utils.data
import cv2
import h5py
from tqdm.auto import tqdm
import os
import numpy
import pandas
import random
import sklearn.preprocessing
import sklearn.model_selection
from importables.preprocessing_managers import PreprocessingManagers, DatasetDictionary, MaskGeneratorMocker, MotionVectorProcessorMocker
from importables.utilities import Utilities
from importables.custom_types import List, Tuple, Any, ndarray, Tensor, Union, MotionVectorFrame, OpticalFlowFrame, SegmentationMask, BoundingBoxXY1XY2, ColorXY
from torchvision import transforms
from importables.constants import SELECTED_NTU_DAILY_ACTIONS_SET, RANDOM_SEED_BYTES

/home/nicholassv/anaconda3/envs/thesis_linux_3.9/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from typing import Any


class FlowDataset(torch.utils.data.Dataset):
    def __init__(
                self, 
                dataset_mapping: dict, 
                label_encoder: sklearn.preprocessing.LabelEncoder, 
                loader: PreprocessingManagers.Loader, 
                dataframe: pandas.DataFrame, 
                h5py_base_file: h5py.File,  
                timestep: int, 
                transform: Union[callable, None] = None, # type: ignore
                label_key: str = "A",
                with_pre_padding: bool = False
        ):
        
        self.__timestep: int = timestep
        self.__label_key: str = label_key
        self.__h5py_base_file: h5py.File = h5py_base_file
        self.__dataset_mapping: dict = dataset_mapping
        self.__index_mapping: List[Tuple[str,int,int,int]] = []
        self.__loader: PreprocessingManagers.Loader = loader
        self.__transform:  Union[callable, None] = transform # type: ignore
        self.__label_encoder: sklearn.preprocessing.LabelEncoder = label_encoder
        self.__with_pre_padding: bool = with_pre_padding

        for index in tqdm(range(len(dataframe)), desc="Generating index mapping..."):

            current_series: pandas.Series = dataframe.iloc[index]
            label: int = current_series[self.__dataset_mapping[self.__label_key]] # type: ignore
            frame_count: int = current_series[self.__dataset_mapping[DatasetDictionary.FRAME_COUNT_KEY]] # type: ignore
            file_path: str = current_series[self.__dataset_mapping[DatasetDictionary.GENERATED_FILENAME_KEY]] # type: ignore

            for frame_index in range(frame_count):
                start_index: int = frame_index - self.__timestep + 1
                stop_index: int = frame_index + 1
                
                if not self.__with_pre_padding:
                    if start_index < 0:
                        continue
                    
                self.__index_mapping.append(
                        (
                            file_path,
                            label,
                            start_index,
                            stop_index,
                        )
                    )
                
    def __len__(self) -> int:
        return len(self.__index_mapping)

    def __getitem__(self, idx: Any) -> dict:
        if torch.is_tensor(idx):
            idx = idx.tolist()

        file_path, label, start_index, stop_index = self.__index_mapping[idx]

        file_group: h5py.Group = self.__h5py_base_file[file_path] # type: ignore
        _, mask_generator, motion_vector_processor, optical_flow_generator = self.__loader.load_from_h5(file_group, start_index, stop_index) 

        label_label: ndarray = self.__label_encoder.transform([[label]])[0] # type: ignore
        returns_list: List[dict[str, Union[ndarray, Tensor]]] = []

        motion_vector_output: bool = motion_vector_processor is not None
        optical_flow_output: bool = optical_flow_generator is not None

        if self.__with_pre_padding:
            if len(mask_generator) < self.__timestep:
                padding_returns: dict[str, Union[ndarray, Tensor]] = {}
                padding_returns["frame_label"] = numpy.zeros_like(label_label)

                if motion_vector_output:
                    padding_returns["motion_vector"] = motion_vector_processor.generate_blank()

                if optical_flow_output:
                    padding_returns["optical_flow"] = optical_flow_generator.generate_blank()

                if self.__transform:
                    padding_returns = self.__transform(padding_returns)

                returns_list = [padding_returns] * int(self.__timestep-len(mask_generator))

        while len(mask_generator) > 0:
            current_frame_returns: dict[str, Union[ndarray, Tensor]] = {}
            current_frame_returns["frame_label"] = numpy.zeros_like(label_label)

            is_mask_available, segmentation_mask, bounding_box = mask_generator.forward_once_with_mcbb()
            if is_mask_available:
                current_frame_returns["frame_label"] = label_label
                current_frame_returns["segmentation_mask"] = segmentation_mask
                current_frame_returns["bounding_box"] = bounding_box

            if motion_vector_output:
                motion_vector_frame: MotionVectorFrame = motion_vector_processor.process()
                current_frame_returns["motion_vector"] = motion_vector_frame

            if optical_flow_output:
                optical_flow_frame: OpticalFlowFrame = optical_flow_generator.forward_once_auto()
                current_frame_returns["optical_flow"] = optical_flow_frame

            if self.__transform:
                current_frame_returns = self.__transform(current_frame_returns)

            if "segmentation_mask" in current_frame_returns:
                del current_frame_returns["segmentation_mask"]
                del current_frame_returns["bounding_box"]

            returns_list.append(current_frame_returns)

        collated_timestep: dict[str, Tensor] = torch.utils.data.default_collate(returns_list)
        collated_timestep["label"] = torch.tensor(label_label)

        return collated_timestep
    
class FlowMaskCropTransform(object):
    def __init__(self, mask: bool = True, crop: bool = True, replace_with: ColorXY = (128,128)) -> None:
        self.__mask: bool = mask
        self.__crop: bool = crop
        self.__replace_with: ColorXY = replace_with
    
    def __call__(self, X: dict) -> dict:
        if "segmentation_mask" not in X:
            return X
        
        segmentation_mask: SegmentationMask = X["segmentation_mask"]
        bounding_box: BoundingBoxXY1XY2 = X["bounding_box"]
        
        if self.__mask:
            if "motion_vector" in X:
                X["motion_vector"][~segmentation_mask] = self.__replace_with

            if "optical_flow" in X:
                X["optical_flow"][~segmentation_mask] = self.__replace_with
            
        if self.__crop:
            if "motion_vector" in X:
                X["motion_vector"] = Utilities.crop_to_bb(X["motion_vector"], bounding_box)

            if "optical_flow" in X:
                X["optical_flow"] = Utilities.crop_to_bb(X["optical_flow"], bounding_box)

        del X["segmentation_mask"]
        del X["bounding_box"]

        return X
    
class FlowPadResize(object):
    def __init__(self, output_size: int, pad_with: ColorXY = (128,128)) -> None:
        self.__output_size: int = output_size
        self.__pad_with: ColorXY = pad_with

    def __call__(self, X: dict) -> dict:
        
        if "motion_vector" in X:
            X["motion_vector"],_,_,_,_,_ = Utilities.letterbox(X["motion_vector"], self.__output_size, self.__pad_with, 0, True)

        if "optical_flow" in X:
            X["optical_flow"],_,_,_,_,_ = Utilities.letterbox(X["optical_flow"], self.__output_size, self.__pad_with, 0, True)

        return X
        
class FlowToCHW(object):

    def __call__(self, X: dict) -> dict:
        
        if "motion_vector" in X:
            X["motion_vector"] = X["motion_vector"].transpose((2, 0, 1))

        if "optical_flow" in X:
            X["optical_flow"] =  X["optical_flow"].transpose((2, 0, 1))

        return X
    
class FlowRescale(object):
    
    def __init__(self, scale: float = 1.0) -> None:
        self.__scale: float = scale

    def __call__(self, X: dict) -> dict:
         
        if "motion_vector" in X:
            X["motion_vector"] = (X["motion_vector"] * self.__scale).astype(numpy.float32)

        if "optical_flow" in X:
            X["optical_flow"] = (X["optical_flow"] * self.__scale).astype(numpy.float32)

        return X

In [3]:
def set_all_seed():
    random.seed(RANDOM_SEED_BYTES)
    numpy.random.seed(int.from_bytes(RANDOM_SEED_BYTES[:4], "big"))
    torch.manual_seed(int.from_bytes(RANDOM_SEED_BYTES[:4], "big"))

In [4]:
curr_mapping = DatasetDictionary.Mappings.NTU_ACTION_RECOGNITION_DATASET
pd_pregen = DatasetDictionary("/mnt/c/Skripsi/dataset-pregen/generated_dictionary.csv",DatasetDictionary.Mappings.NTU_ACTION_RECOGNITION_DATASET).as_DataFrame()
pd_pregen_filter = pd_pregen.loc[pd_pregen[curr_mapping["A"]].isin(list(SELECTED_NTU_DAILY_ACTIONS_SET)[:5])]
pd_pregen_filter = pd_pregen_filter.reset_index(drop = True)

In [5]:
pd_pregen_filter

,ActionID,CameraID,PerformerID,ReplicationID,SetupID,SourceFilePath,GeneratedFileName,FrameCount
0,2,1,2,1,1,../../dataset-h264/R001A002/S001C001P002R001A0...,S001C001P002R001A002_rgb.h5,88
1,2,1,3,1,1,../../dataset-h264/R001A002/S001C001P003R001A0...,S001C001P003R001A002_rgb.h5,110
2,2,1,4,1,1,../../dataset-h264/R001A002/S001C001P004R001A0...,S001C001P004R001A002_rgb.h5,133
3,2,1,5,1,1,../../dataset-h264/R001A002/S001C001P005R001A0...,S001C001P005R001A002_rgb.h5,64
4,2,1,6,1,1,../../dataset-h264/R001A002/S001C001P006R001A0...,S001C001P006R001A002_rgb.h5,73
...,...,...,...,...,...,...,...,...
4758,70,3,102,2,32,../../dataset-h264/R002A070/S032C003P102R002A0...,S032C003P102R002A070_rgb.h5,53
4759,70,3,103,2,32,../../dataset-h264/R002A070/S032C003P103R002A0...,S032C003P103R002A070_rgb.h5,46
4760,70,3,104,2,32,../../dataset-h264/R002A070/S032C003P104R002A0...,S032C003P104R002A070_rgb.h5,31
4761,70,3,105,2,32,../../dataset-h264/R002A070/S032C003P105R002A0...,S032C003P105R002A070_rgb.h5,48


In [6]:
pd_pregen_filter[curr_mapping["A"]].unique()

<IntegerArray>
[2, 16, 20, 68, 70]
Length: 5, dtype: Int64

In [7]:
pregen_enc = sklearn.preprocessing.LabelEncoder().fit(numpy.expand_dims(pd_pregen_filter[curr_mapping["A"]].astype(int).unique(),1))
pregen_enc.classes_

array([ 2, 16, 20, 68, 70])

In [8]:
pregen_enc.transform([[2]])

array([0])

In [9]:
# use only subset of data
set_all_seed()
_, pd_pregen_filter = sklearn.model_selection.train_test_split(pd_pregen_filter, test_size =0.1, shuffle=True, stratify=pd_pregen_filter[curr_mapping["A"]])


In [10]:
set_all_seed()
train_pd, test_pd = sklearn.model_selection.train_test_split(pd_pregen_filter, test_size =0.3, shuffle=True, stratify=pd_pregen_filter[curr_mapping["A"]])

In [11]:
train_pd.reset_index(drop = True)
test_pd.reset_index(drop = True)

,ActionID,CameraID,PerformerID,ReplicationID,SetupID,SourceFilePath,GeneratedFileName,FrameCount
0,20,3,1,2,1,../../dataset-h264/R002A020/S001C003P001R002A0...,S001C003P001R002A020_rgb.h5,104
1,68,2,75,1,30,../../dataset-h264/R001A068/S030C002P075R001A0...,S030C002P075R001A068_rgb.h5,76
2,2,3,25,2,8,../../dataset-h264/R002A002/S008C003P025R002A0...,S008C003P025R002A002_rgb.h5,79
3,16,2,18,2,12,../../dataset-h264/R002A016/S012C002P018R002A0...,S012C002P018R002A016_rgb.h5,97
4,2,2,7,2,12,../../dataset-h264/R002A002/S012C002P007R002A0...,S012C002P007R002A002_rgb.h5,119
...,...,...,...,...,...,...,...,...
139,68,1,41,1,28,../../dataset-h264/R001A068/S028C001P041R001A0...,S028C001P041R001A068_rgb.h5,41
140,20,3,19,1,9,../../dataset-h264/R001A020/S009C003P019R001A0...,S009C003P019R001A020_rgb.h5,65
141,16,3,23,1,6,../../dataset-h264/R001A016/S006C003P023R001A0...,S006C003P023R001A016_rgb.h5,217
142,2,2,19,1,10,../../dataset-h264/R001A002/S010C002P019R001A0...,S010C002P019R001A002_rgb.h5,75


In [12]:
train_pd.ActionID.value_counts()

70    67
68    67
2     67
16    66
20    66
Name: ActionID, dtype: Int64

In [13]:
test_pd.ActionID.value_counts()

20    29
68    29
16    29
70    29
2     28
Name: ActionID, dtype: Int64

In [14]:
h5py_instance = h5py.File("/mnt/c/Skripsi/dataset-pregen/generated_consolidation.h5", swmr=True, rdcc_nbytes=1024**2*4000, rdcc_nslots=1e7)
args_mvprocessor = {
    "raw_motion_vectors": False,
    "target_size": 418,
}

args_maskgenerator = {
    "weight_path": './libs/yolov7-mask/yolov7-mask.pt',
    "hyperparameter_path": './libs/yolov7-mask/data/hyp.scratch.mask.yaml',
    "confidence_threshold": 0.5,
    "iou_threshold": 0.45,
    "target_size": 418,
    "optimize_model": True,
    "bounding_box_grouping_range_scale": 10
}

loader = PreprocessingManagers.Loader(DatasetDictionary.Mappings.NTU_ACTION_RECOGNITION_DATASET, args_maskgenerator, args_mvprocessor, None)
timestep = 24

In [15]:
train_ds = FlowDataset(
    DatasetDictionary.Mappings.NTU_ACTION_RECOGNITION_DATASET,
    pregen_enc,
    loader,
    train_pd,
    h5py_instance,
    timestep,
    transforms.Compose([
        FlowMaskCropTransform(),
        FlowPadResize(224),
        FlowToCHW(),
        FlowRescale(1/255.)
    ])
    )

test_ds = FlowDataset(
    DatasetDictionary.Mappings.NTU_ACTION_RECOGNITION_DATASET,
    pregen_enc,
    loader,
    test_pd,
    h5py_instance,
    timestep,
    transforms.Compose([
        FlowMaskCropTransform(),
        FlowPadResize(224),
        FlowToCHW(),
        FlowRescale(1/255.)
    ])
    )

Generating index mapping...:   0%|          | 0/333 [00:00<?, ?it/s]

Generating index mapping...:   0%|          | 0/144 [00:00<?, ?it/s]

In [16]:
train_ds[0]

{'frame_label': tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
 'motion_vector': tensor([[[[0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           ...,
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020]],
 
          [[0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           ...,
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
           [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020]]],
 
 
         [[[0.5020, 0.5020, 0.5020,  ...

In [17]:
# for i in tqdm(train_ds):
#     i

In [18]:
BATCH_SIZE = 16
NUM_WORKERS = torch.get_num_threads()
train_dl = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, num_workers=3, shuffle= True)
test_dl = torch.utils.data.DataLoader(test_ds, BATCH_SIZE, num_workers=3, shuffle= True)

In [19]:
for x in tqdm(train_dl):
    print(x)
    break

  0%|          | 0/1277 [00:00<?, ?it/s]

{'frame_label': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],


In [20]:
import torchvision.models
import torch.functional

In [21]:
# resnet18_model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
# resnet18_model

In [22]:
# resnet18_model.conv1 = torch.nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False)
# resnet18_model.fc = torch.nn.Identity()
# resnet18_model

In [23]:
for x in train_dl:
    print(x["motion_vector"][:,0,...].shape)
    # print(resnet18_model(x["motion_vector"][:,0,...]))
    break

torch.Size([16, 2, 224, 224])


In [24]:
# resnet18_model(x["motion_vector"][:,0,...]).shape

In [25]:
# class HARMVModelCNNLSTMv1(torch.nn.Module):
#     def __init__(self, category_count: int, timesteps: int, lstm_hidden: int = 64) -> None:
#         super(HARMVModelCNNLSTMv1, self).__init__()
#         self.__lstm_hidden: int = lstm_hidden
#         self.__timesteps: int = timesteps
#         self.resnet18_model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
#         self.resnet18_model.conv1 = torch.nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False)
#         self.resnet18_model.fc = torch.nn.Identity()
#         self.resnet18_model = self.resnet18_model.requires_grad_(False)
#         self.resnet18_model = self.resnet18_model.eval()
#         self.lstm1 = torch.nn.LSTM(512,  self.__lstm_hidden, bidirectional=True, batch_first=True)
#         self.lstm2 = torch.nn.LSTM( self.__lstm_hidden * 2,  self.__lstm_hidden, bidirectional=True, batch_first=True)
#         self.fc1_frame = torch.nn.Linear( self.__lstm_hidden*2,  self.__lstm_hidden*2)
#         self.fc2_frame = torch.nn.Linear( self.__lstm_hidden*2, category_count)
#         self.fc1_global = torch.nn.Linear( self.__lstm_hidden * 2*self.__timesteps,  self.__lstm_hidden*2)
#         self.fc2_global = torch.nn.Linear( self.__lstm_hidden*2, category_count)

#     def forward(self, x: Tensor):
#         batch_size, timesteps, C, H, W = x.shape
#         assert self.__timesteps == timesteps, f"Timesteps not equal to {self.__timesteps}"
#         with torch.no_grad():
#             x = x.view(batch_size * self.__timesteps, C, H, W)
#             x = self.resnet18_model(x)
#             x = x.view(batch_size, self.__timesteps, -1)
            
#         self.lstm1.flatten_parameters()
#         x, _, _ = self.lstm1(x)
#         self.lstm2.flatten_parameters()
#         x, _, _ = self.lstm2(x)
#         x = x.contiguous()

#         x_frame = x.view(batch_size * self.__timesteps,  self.__lstm_hidden * 2)
#         x_frame = torch.nn.functional.relu(self.fc1_frame(x_frame))
#         x_frame = torch.nn.functional.softmax(self.fc2_frame(x_frame), dim=1)
#         x_frame = x_frame.view(batch_size, timesteps, -1)

#         x_global = x.view(batch_size, -1)
#         x_global = torch.nn.functional.relu(self.fc1_global(x_global))
#         x_global = torch.nn.functional.softmax(self.fc2_global(x_global), dim=1)

#         return {"frame": x_frame, "global": x_global}


class HARMVModelCNNLSTMv1(torch.nn.Module):
    def __init__(self, category_count: int, timesteps: int, lstm_hidden: int = 64) -> None:
        super(HARMVModelCNNLSTMv1, self).__init__()
        self.__lstm_hidden: int = lstm_hidden
        self.__timesteps: int = timesteps
        self.resnet18_model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
        self.resnet18_model.conv1 = torch.nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet18_model.fc = torch.nn.Identity()
        self.resnet18_model = self.resnet18_model.requires_grad_(False)
        self.resnet18_model = self.resnet18_model.eval()
        self.lstm1 = torch.nn.LSTM(512,  self.__lstm_hidden, bidirectional=True, batch_first=True)
        self.lstm2 = torch.nn.LSTM( self.__lstm_hidden * 2,  self.__lstm_hidden, bidirectional=True, batch_first=True)
        self.fc1 = torch.nn.Linear( self.__lstm_hidden*2,  self.__lstm_hidden*4)
        self.fc2 = torch.nn.Linear( self.__lstm_hidden*4,  self.__lstm_hidden*2)
        self.fc3 = torch.nn.Linear( self.__lstm_hidden*2, category_count)

    def forward(self, x: Tensor):
        batch_size, timesteps, C, H, W = x.shape
        assert self.__timesteps == timesteps, f"Timesteps not equal to {self.__timesteps}"
        with torch.no_grad():
            x = x.view(batch_size * self.__timesteps, C, H, W)
            x = self.resnet18_model(x)
            x = x.view(batch_size, self.__timesteps, -1)
            
        self.lstm1.flatten_parameters()
        lo, (x, _) = self.lstm1(x)
        self.lstm2.flatten_parameters()
        _, (x, _) = self.lstm2(lo)

        x = torch.cat((x[0],x[1]), dim = 1)

        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)

        return x


In [26]:
test_model = HARMVModelCNNLSTMv1(len(pregen_enc.classes_), timestep, 128)
test_model = test_model.to("cuda:0")

In [27]:
import torchinfo
torchinfo.summary(test_model,input_data=x["motion_vector"])

Layer (type:depth-idx)                        Output Shape              Param #
HARMVModelCNNLSTMv1                           [16, 5]                   --
├─ResNet: 1-1                                 [384, 512]                --
│    └─Conv2d: 2-1                            [384, 64, 112, 112]       (6,272)
│    └─BatchNorm2d: 2-2                       [384, 64, 112, 112]       (128)
│    └─ReLU: 2-3                              [384, 64, 112, 112]       --
│    └─MaxPool2d: 2-4                         [384, 64, 56, 56]         --
│    └─Sequential: 2-5                        [384, 64, 56, 56]         --
│    │    └─BasicBlock: 3-1                   [384, 64, 56, 56]         (73,984)
│    │    └─BasicBlock: 3-2                   [384, 64, 56, 56]         (73,984)
│    └─Sequential: 2-6                        [384, 128, 28, 28]        --
│    │    └─BasicBlock: 3-3                   [384, 128, 28, 28]        (230,144)
│    │    └─BasicBlock: 3-4                   [384, 128, 28, 28]    

In [28]:
# test_model(x["motion_vector"].to("cuda"))

In [29]:
# set_all_seed()
# from IPython.display import clear_output
# import torch.utils.tensorboard.writer
# writer = torch.utils.tensorboard.writer.SummaryWriter()
# criterion_frame = torch.nn.CrossEntropyLoss()
# criterion_global = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(test_model.parameters(), lr=0.001)
# EPOCH = 2
# for e in range(EPOCH):
#     for data in tqdm(train_dl):
#         inputs = data["motion_vector"].to("cuda")
#         frame_labels = data["frame_label"].float().to("cuda").view(BATCH_SIZE*timestep,-1)
#         global_labels = data["label"].float().to("cuda")

#         optimizer.zero_grad()
#         outputs = test_model(inputs)
#         output_frame = outputs["frame"].view(BATCH_SIZE*timestep,-1)
#         output_global = outputs["global"]
#         loss_frame = criterion_frame(output_frame, frame_labels)
#         loss_global = criterion_global(output_global, global_labels)
#         writer.add_scalar("Loss-frame/train", loss_frame, e)
#         writer.add_scalar("Loss-global/train", loss_global, e)
#         sum_loss = loss_frame + loss_global
#         sum_loss.backward()
#         optimizer.step()
#         output_frame_b = (output_frame>0.5).float()
#         correct_frame = (output_frame_b == frame_labels).float().sum()
#         output_global_b = (output_global>0.5).float()
#         correct_global = (output_global_b == global_labels).float().sum()
#         writer.add_scalar("Accuracy-frame/train", correct_frame/(BATCH_SIZE*timestep), e)
#         writer.add_scalar("Accuracy-global/train", correct_global/BATCH_SIZE, e)
#         clear_output(wait=True)
#         print()
#         print(f"Epoch {e} ================")
#         print("Loss f|g : ({:.3f}|{:.3f})".format(loss_frame, loss_global))
#         print("Acc f|g : ({:.3f}|{:.3f})".format(correct_frame/(BATCH_SIZE*timestep), correct_global/BATCH_SIZE))
#         print("==========================")


set_all_seed()
import torch.utils.tensorboard.writer
writer = torch.utils.tensorboard.writer.SummaryWriter()
criterion_global = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(test_model.parameters(), lr=0.01)
EPOCH = 10
for e in range(EPOCH):
    with tqdm(total=len(train_dl)) as pbar:
        loss_sum_avg = []
        acc_sum_avg = []
        for data in train_dl:
            inputs = data["motion_vector"].to("cuda")
            global_labels = data["label"].to("cuda")

            optimizer.zero_grad()
            output_global = test_model(inputs)
            loss_global = criterion_global(output_global, global_labels)
            loss_global.backward()
            optimizer.step()

            with torch.no_grad():
                correct_global = (torch.argmax(output_global.softmax(dim=1), dim=1) == global_labels).float().sum()

                writer.add_scalar("Batch-Loss-global/train", loss_global.item(), e*len(train_dl)+len(loss_sum_avg))
                writer.add_scalar("Batch-Accuracy-global/train", correct_global/BATCH_SIZE, e*len(train_dl)+len(loss_sum_avg))

                loss_sum_avg.append(loss_global.item())
                acc_sum_avg.append(correct_global.item()/BATCH_SIZE)

            pbar.set_description("Training: Epoch {} | Loss: {:.3f} | Accuracy {:.3f}".format(e, sum(loss_sum_avg)/len(loss_sum_avg),  sum(acc_sum_avg)/len(acc_sum_avg)))
            pbar.update()

        writer.add_scalar("Epoch-Loss-global/train", sum(loss_sum_avg)/len(loss_sum_avg), e)
        writer.add_scalar("Epoch-Accuracy-global/train", sum(acc_sum_avg)/len(acc_sum_avg), e)

    with tqdm(total=len(test_dl)) as pbar:
        loss_sum_avg = []
        acc_sum_avg = []
        for data in test_dl:
            with torch.no_grad():
                inputs = data["motion_vector"].to("cuda")
                global_labels = data["label"].to("cuda")

                output_global = test_model(inputs)
                loss_global = criterion_global(output_global, global_labels)
                correct_global = (torch.argmax(output_global.softmax(dim=1), dim=1) == global_labels).float().sum()

                writer.add_scalar("Batch-Loss-global/test", loss_global.item(), e*len(test_dl)+len(loss_sum_avg))
                writer.add_scalar("Batch-Accuracy-global/test", correct_global/BATCH_SIZE, e*len(test_dl)+len(loss_sum_avg))

                loss_sum_avg.append(loss_global.item())
                acc_sum_avg.append(correct_global.item()/BATCH_SIZE)

                pbar.set_description("Validation: Epoch {} | Loss: {:.3f} | Accuracy {:.3f}".format(e, sum(loss_sum_avg)/len(loss_sum_avg),  sum(acc_sum_avg)/len(acc_sum_avg)))
                pbar.update()

        writer.add_scalar("Epoch-Loss-global/test", sum(loss_sum_avg)/len(loss_sum_avg), e)
        writer.add_scalar("Epoch-Accuracy-global/test", sum(acc_sum_avg)/len(acc_sum_avg), e)

  0%|          | 0/1277 [00:00<?, ?it/s]

  0%|          | 0/581 [00:00<?, ?it/s]

  0%|          | 0/1277 [00:00<?, ?it/s]

  0%|          | 0/581 [00:00<?, ?it/s]

  0%|          | 0/1277 [00:00<?, ?it/s]

KeyboardInterrupt: 